In [31]:
import pandas as pd

In [32]:
tabela_1 = pd.DataFrame(data={
    'col1': [1, 2],
    'col2': [3, 4]
})

In [33]:
tabela_2 = pd.DataFrame(data={
    'cod_fornecedor': ['F-201', 'F-305', 'F-118', 'F-410'],
    'razao_social': ['Distribuidora Elétrica São Paulo Ltda', 'Eletropeças Mineira ME', 'Casa do Eletricista Conceição & Filhos', 'Iluminação Nordeste S.A.'],
    'num_contrato': ['CTR-2024-001', 'CTR-2024-015', 'CTR-2024-022', 'CTR-2024-033'],
    'vigencia_inicio': ['31/12/2025', '28/02/2026', '14/06/2026', '31/08/2026'],
    'valor_contrato': [250000.00, 180000.00, 320000.00, 95000.00]
})

In [34]:
tabela_2.dtypes

cod_fornecedor         str
razao_social           str
num_contrato           str
vigencia_inicio        str
valor_contrato     float64
dtype: object

In [35]:
tabela_3 = pd.DataFrame(data={
    'col1': [1, 2],
    'col2': [3, 4]
})

In [36]:
tabela_3

,col1,col2
0,1,3
1,2,4


In [37]:
tabela_1.to_html('tabela_1.html', index=False)
tabela_2.to_html('tabela_2.html', index=False)
tabela_3.to_html('tabela_3.html', index=False)

In [91]:
pd.read_html('tabela_1.html')[1]

IndexError: list index out of range

In [ ]:
dados_contratos_ativos = pd.read_html('contratos_ativos.html')[1]
dados_notas_fiscais = pd.read_xml('notas_fiscais.xml')

In [107]:
dados_contratos_ativos['vigencia_inicio'] = dados_contratos_ativos['vigencia_inicio'].astype('datetime64[ns]')

In [108]:
dados_contratos_ativos.dtypes

cod_fornecedor                str
razao_social                  str
num_contrato                  str
vigencia_inicio    datetime64[ns]
valor_contrato            float64
dtype: object

In [109]:
dados_notas_fiscais.dtypes

numero_nf                    str
cod_fornecedor               str
data_emissao      datetime64[ms]
valor_nf                 float64
dtype: object

In [110]:
dados_notas_fiscais['data_emissao'] = dados_notas_fiscais['data_emissao'].astype('datetime64[ns]')

In [105]:
dados_notas_fiscais.dtypes

numero_nf                    str
cod_fornecedor               str
data_emissao      datetime64[ms]
valor_nf                 float64
dtype: object

In [120]:
dados_notas_fiscais_agrupadas = dados_notas_fiscais.groupby(by='cod_fornecedor', as_index=False).agg(
    numero_nf = pd.NamedAgg(column='numero_nf', aggfunc='sum'),
    data_emissao = pd.NamedAgg(column='data_emissao', aggfunc=list),
    total_faturado = pd.NamedAgg(column='valor_nf', aggfunc='sum'),
    qtd_notas = pd.NamedAgg(column='numero_nf', aggfunc='count')
)

In [118]:
dados_notas_fiscais_agrupadas

,cod_fornecedor,numero_nf,data_emissao,total_faturado,qtd_notas
0,F-118,NF-10118NF-10119NF-10120,2025-02-12,164000.0,3
1,F-201,NF-10201NF-10202,2025-02-10,77000.0,2
2,F-305,NF-10305,2025-01-20,28000.0,1
3,F-500,NF-10500,2025-02-01,12000.0,1


In [73]:
dados_consolidados = dados_contratos_ativos.merge(dados_notas_fiscais_agrupadas, how='outer', on='cod_fornecedor')

In [74]:
dados_consolidados

,cod_fornecedor,razao_social,num_contrato,vigencia_inicio,valor_contrato,numero_nf,data_emissao,total_faturado,qtd_notas
0,F-118,Casa do Eletricista ConceiÃ§Ã£o & Filhos,CTR-2024-022,14/06/2026,320000.0,NF-10118NF-10119NF-10120,2025-01-252025-02-082025-02-12,164000.0,3.0
1,F-201,Distribuidora ElÃ©trica SÃ£o Paulo Ltda,CTR-2024-001,31/12/2025,250000.0,NF-10201NF-10202,2025-01-152025-02-10,77000.0,2.0
2,F-305,EletropeÃ§as Mineira ME,CTR-2024-015,28/02/2026,180000.0,NF-10305,2025-01-20,28000.0,1.0
3,F-410,IluminaÃ§Ã£o Nordeste S.A.,CTR-2024-033,31/08/2026,95000.0,NaN,NaN,NaN,NaN
4,F-500,NaN,NaN,NaN,NaN,NF-10500,2025-02-01,12000.0,1.0


In [75]:
dados_consolidados[['total_faturado', 'qtd_notas']] = dados_consolidados[['total_faturado', 'qtd_notas']].fillna(0)

In [77]:
dados_consolidados[['razao_social']] = dados_consolidados[['razao_social']].fillna('SEM CONTRATO')

In [78]:
dados_consolidados

,cod_fornecedor,razao_social,num_contrato,vigencia_inicio,valor_contrato,numero_nf,data_emissao,total_faturado,qtd_notas
0,F-118,Casa do Eletricista ConceiÃ§Ã£o & Filhos,CTR-2024-022,14/06/2026,320000.0,NF-10118NF-10119NF-10120,2025-01-252025-02-082025-02-12,164000.0,3.0
1,F-201,Distribuidora ElÃ©trica SÃ£o Paulo Ltda,CTR-2024-001,31/12/2025,250000.0,NF-10201NF-10202,2025-01-152025-02-10,77000.0,2.0
2,F-305,EletropeÃ§as Mineira ME,CTR-2024-015,28/02/2026,180000.0,NF-10305,2025-01-20,28000.0,1.0
3,F-410,IluminaÃ§Ã£o Nordeste S.A.,CTR-2024-033,31/08/2026,95000.0,NaN,NaN,0.0,0.0
4,F-500,SEM CONTRATO,NaN,NaN,NaN,NF-10500,2025-02-01,12000.0,1.0


In [83]:
dados_consolidados['percentual_executado'] = ((dados_consolidados['total_faturado'] / dados_consolidados['valor_contrato']) * 100).round(1)

In [86]:
dados_consolidados['percentual_executado'] = dados_consolidados['percentual_executado'].fillna('N/A')

In [87]:
dados_consolidados

,cod_fornecedor,razao_social,num_contrato,vigencia_inicio,valor_contrato,numero_nf,data_emissao,total_faturado,qtd_notas,percentual_executado
0,F-118,Casa do Eletricista ConceiÃ§Ã£o & Filhos,CTR-2024-022,14/06/2026,320000.0,NF-10118NF-10119NF-10120,2025-01-252025-02-082025-02-12,164000.0,3.0,51.2
1,F-201,Distribuidora ElÃ©trica SÃ£o Paulo Ltda,CTR-2024-001,31/12/2025,250000.0,NF-10201NF-10202,2025-01-152025-02-10,77000.0,2.0,30.8
2,F-305,EletropeÃ§as Mineira ME,CTR-2024-015,28/02/2026,180000.0,NF-10305,2025-01-20,28000.0,1.0,15.6
3,F-410,IluminaÃ§Ã£o Nordeste S.A.,CTR-2024-033,31/08/2026,95000.0,NaN,NaN,0.0,0.0,0.0
4,F-500,SEM CONTRATO,NaN,NaN,NaN,NF-10500,2025-02-01,12000.0,1.0,N/A


In [88]:
dados_consolidados.to_html('auditoria_contratos.html')

In [89]:
dados_consolidados.to_excel('auditoria_contratos.xlsx', sheet_name='auditoria', index=False)

In [90]:
dados_consolidados

,cod_fornecedor,razao_social,num_contrato,vigencia_inicio,valor_contrato,numero_nf,data_emissao,total_faturado,qtd_notas,percentual_executado
0,F-118,Casa do Eletricista ConceiÃ§Ã£o & Filhos,CTR-2024-022,14/06/2026,320000.0,NF-10118NF-10119NF-10120,2025-01-252025-02-082025-02-12,164000.0,3.0,51.2
1,F-201,Distribuidora ElÃ©trica SÃ£o Paulo Ltda,CTR-2024-001,31/12/2025,250000.0,NF-10201NF-10202,2025-01-152025-02-10,77000.0,2.0,30.8
2,F-305,EletropeÃ§as Mineira ME,CTR-2024-015,28/02/2026,180000.0,NF-10305,2025-01-20,28000.0,1.0,15.6
3,F-410,IluminaÃ§Ã£o Nordeste S.A.,CTR-2024-033,31/08/2026,95000.0,NaN,NaN,0.0,0.0,0.0
4,F-500,SEM CONTRATO,NaN,NaN,NaN,NF-10500,2025-02-01,12000.0,1.0,N/A
